In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime

In [2]:
now = datetime.now()
options = Options() 
options.add_experimental_option("excludeSwitches", ['enable-automation']) # allows us to scrape bloomberg
#options.add_argument("--disable-notifications") #just added
options.add_argument("--start-maximized")
tickers = ['ADOM','BKCH','AESEW','AMR','AMRWW','BLU','BIQI','CLDC','CNTF','CCIH','CRAY','DFRG','DVCR','EMMA','EMMAW','FNSR','FRED','GLADN','HUNT','HUNTU','HUNTW','IVENC','IVFGC','IVFVC','SKIS','REED','TBRGU','SFLY','SLNG','USAT','USATP','BCACU','XYN','XYNPW','YRIV']

In [41]:
# gathers list of bloomberg articles for a keyword
# bloomberg search replacement
def fetch_links(link):
    browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver")
    browser.get(link)
    time.sleep(3)
    actions = ActionChains(browser)
    
    browser.find_element_by_css_selector("div.container__0c254cfc > div.container__9dcb6ba7:nth-child(2) > div.listWrap__6cc206d2 > a:nth-child(2)").click()
    time.sleep(1)
    # overlooked
    # must change to find date and stop searching when posts are not from 2019
    # currently, just stops after 5 load more button presses
    max = 5
    counter = 0
    while browser.find_elements_by_css_selector("section.mainContent__b8b5cf5e > div:last-child > div:last-child > button"):
        if counter == max:
            break
        for _ in range(3):   #Adjust this range according to your need, I meant how far you wanna go down.
            actions.send_keys(Keys.SPACE).perform()
            time.sleep(1)
        browser.find_element_by_css_selector("section.mainContent__b8b5cf5e > div:last-child > div:last-child > button").click()
        time.sleep(3)
        counter += 1
        
    links = scrapeSearch(browser.page_source)
    browser.quit()
    
    return links

def get_ticker_links(ticker):
    tick = fetch_links('https://www.bloomberg.com/search?query=%s'%(ticker))
    tick_raw = []
    i = 0
    while i < len(tick):
        articleSource = tick[i][1]['href']
        year = tick[i][0].text
        tick_raw.append([ticker, year, articleSource])
        i+=1
    tick_df = pd.DataFrame(tick_raw, columns=['STOCK_ID', 'YEAR', 'URL'])
    return tick_df
    #df = df.append(tick_df, ignore_index=True)

'''
def bloombergSearch(url):
    #url = 'https://www.bloomberg.com/search?query=%s'%(ticker)
    browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver") # path should be similar for ubuntu vm
    browser.get(url)
    time.sleep(5) # wait for the page to load javascript
    pageSource = browser.page_source
    browser.quit()
    return pageSource
'''

def bloombergArticle(url):
    browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver")
    browser.delete_all_cookies() # bypass bloomberg article limit
    browser.get(url)
    time.sleep(5) # wait for the page to load javascript
    pageSource = browser.page_source
    browser.quit()
    return pageSource

def contentCat(content):
# concatenate content strings of article
    article = ''
    for par in content:
        if par == content[-1]:
            article += par.get_text()
        else:
            article += par.get_text() + '\n'
    return article

def scrapeSearch(pageSource):
    soup = BeautifulSoup(pageSource, 'html.parser')
    searchResults = soup.select('div>div>section:nth-child(3)>div:last-child>div>a', id='root')
    year = soup.select('div>div>section:nth-child(3)>div:last-child>div>div>div.publishedAt__79f8aaad', id='root')
    return list(zip(year,searchResults))

def scrapeArticle(pageSource):
    soup = BeautifulSoup(pageSource, 'html.parser')
    url = soup.select('head>title+link')[0]['href']
    if soup.find_all('h1'):
        title = soup.find_all('h1')[0].get_text()
    elif soup.find('div', {'class':'lede-newsletter__newsletter-slug'}):
        title = soup.find('div', {'class':'lede-newsletter__newsletter-slug'}).text
    else: 
        title = None
    if soup.find_all(class_='author-v2__byline'):    
        author = soup.find_all(class_='author-v2__byline')[0].get_text()
    else:
        author = None
    posted = soup.find_all('time', class_='article-timestamp')[0].get_text() # time of publish
    '''
    if len(soup.find_all('time', class_='article-timestamp')) == 2: 
        edited = soup.find_all('time', class_='article-timestamp')[1].get_text() # if no update date is available, uses publish date
    else: 
        edited = np.nan
    '''
    scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    content = soup.select('div>p', class_='body-copy-v2')
    article = contentCat(content)
    return [url, title, author, posted, scraped, article]

In [42]:
tickers = ['ADOM','BKCH','AESEW','AMR','AMRWW','BLU','BIQI','CLDC','CNTF','CCIH','CRAY','DFRG','DVCR','EMMA','EMMAW','FNSR','FRED','GLADN','HUNT','HUNTU','HUNTW','IVENC','IVFGC','IVFVC','SKIS','REED','TBRGU','SFLY','SLNG','USAT','USATP','BCACU','XYN','XYNPW','YRIV']

In [45]:
links_df = pd.DataFrame(columns=['STOCK_ID', 'YEAR', 'URL'])
# if this breaks you need to adjust ticker range
for ticker in tickers:
    ticker_df = get_ticker_links(ticker)
    links_df = links_df.append(ticker_df, ignore_index=True)

/home/albert/anaconda3/envs/cs/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [46]:
# transfrom date to get int year
links_df['YEAR'] = links_df['YEAR'].apply(lambda x: x[-4:])
links_df['YEAR'] = links_df['YEAR'].apply(lambda x: int(x))

In [47]:
# get only 2019 links
df_2019 = links_df[links_df['YEAR'] == 2019]

In [65]:
remove_videos = ~df_2019['URL'].str.contains('videos')

In [66]:
df_2019 = df_2019[remove_videos]

In [71]:
df_2019 = df_2019.reset_index(drop=True)

In [74]:
bloomberg_df = pd.DataFrame(columns=['URL', 'HEADLINE', 'AUTHOR', 'DATE_OF_PUBLISH', 'DATE_OF_SCRAPE', 'CONTENT'])

In [153]:
# will break time to time, must reset range where index stopped appending
for val in df_2019['URL'].values[79:100]:
    source = bloombergArticle(val)
    data = pd.DataFrame([scrapeArticle(source)], columns=['URL', 'HEADLINE', 'AUTHOR', 'DATE_OF_PUBLISH', 'DATE_OF_SCRAPE', 'CONTENT'])
    bloomberg_df = bloomberg_df.append(data, ignore_index=True)

/home/albert/anaconda3/envs/cs/lib/python3.7/site-packages/ipykernel_launcher.py:53: DeprecationWarning: use options instead of chrome_options


WebDriverException: Message: chrome not reachable
  (Session info: chrome=77.0.3865.90)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 5.0.0-31-generic x86_64)


In [159]:
merge_df = pd.concat([links_df, bloomberg_df], axis=1)

In [160]:
merge_df

,STOCK_ID,YEAR,URL,URL,HEADLINE,AUTHOR,DATE_OF_PUBLISH,DATE_OF_SCRAPE,CONTENT
0,ADOM,2019,https://www.bloomberg.com/news/articles/2019-0...,https://www.bloomberg.com/news/articles/2019-0...,Blast in West Bank Kills 17-Year-Old Israeli G...,Ivan Levingston,"August 23, 2019, 4:46 AM EDT",08/10/2019 11:38:23,Israeli soldiers at the scene of the attack in...
1,ADOM,2019,https://www.bloomberg.com/news/articles/2019-0...,https://www.bloomberg.com/news/articles/2019-0...,Israel Launches Strikes Across Gaza After Day ...,Gwen Ackerman,"March 25, 2019, 1:29 AM EDT",08/10/2019 11:38:23,Israel carried out dozens of airstrikes across...
2,AMR,2019,https://www.bloomberg.com/news/articles/2019-0...,https://www.bloomberg.com/news/articles/2019-0...,Saudis Flex Their Patriotic Muscle After Drone...,Donna Abu-Nasr,"September 20, 2019, 8:18 AM EDT",08/10/2019 11:38:23,A festival of concerts and fireworks for the k...
3,AMR,2019,https://www.bloomberg.com/news/articles/2019-0...,https://www.bloomberg.com/news/articles/2019-0...,Have We Hit Peak Supercar?,Hannah Elliott,"September 3, 2019, 4:15 AM EDT",08/10/2019 11:38:23,In a sea of electric concepts and far-off prot...
4,AMR,2019,https://www.bloomberg.com/news/newsletters/201...,https://www.bloomberg.com/news/newsletters/201...,NaN,NaN,"August 21, 2019, 5:01 PM EDT",08/10/2019 11:38:23,"The world is a chaotic mess, but apparently th..."
...,...,...,...,...,...,...,...,...,...
74,EMMA,2019,https://www.bloomberg.com/news/articles/2019-1...,https://www.bloomberg.com/news/articles/2019-1...,Banksy Sets Auction Record With Chimpanzees in...,Katya Kazakina,"October 3, 2019, 3:27 PM EDT",08/10/2019 11:38:23,A painting depicting members of the British Pa...
75,EMMA,2019,https://www.bloomberg.com/news/articles/2019-1...,https://www.bloomberg.com/news/articles/2019-1...,"Biden’s $15.2 Million Trails Sanders, Buttigie...",Tyler Pager,"October 3, 2019, 6:00 AM EDT",08/10/2019 11:38:23,Source: Bloomberg\nSource: Bloomberg\nJoe Bide...
76,EMMA,2019,https://www.bloomberg.com/news/articles/2019-0...,https://www.bloomberg.com/news/articles/2019-0...,"From Washington to Hong Kong, Defiance Abounds...",Kathleen Hunter,"September 28, 2019, 8:00 AM EDT",08/10/2019 11:38:23,U.S. President Donald Trump leaves an event af...
77,EMMA,2019,https://www.bloomberg.com/news/articles/2019-0...,https://www.bloomberg.com/news/articles/2019-0...,Jailed Banker’s Wife Won’t Be Extradited From ...,Jonathan Browning,"September 26, 2019, 11:10 AM EDT",08/10/2019 11:38:23,"The wife of a jailed banker, the target of the..."


In [163]:
df = merge_df[['STOCK_ID', 'URL', 'DATE_OF_PUBLISH', 'HEADLINE', 'CONTENT']]

In [164]:
df.to_csv('bloomberg_articles.csv', encoding='utf-8')